<a href="https://colab.research.google.com/github/ranmeus/TR-PDF_Converter/blob/main/TradeRepublic_PDF_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0.4 | TradeRepublic Portfolio**
# *PDF Converter for Portfolio Performance & Investing.com*
# a) Description
This tool should help to keep the overview of the transactions within TradeRepublic. In TR there is the possibility to export all orders as PDF. For testing purposes the import and export is currently only possible with G-Drive. Alternatives are on the todo list.


## Output A: Master Sheet
The master sheet lists all previous transactions from the PDFs.

## Output B: Delta Sheet
The delta sheet contains all new transactions that have been added since the script was last executed. These are intended to be imported into Portfolio Performance and Investing.com. Delta sheets are dated.

## Output C: Portfolio Sheet
The portfolio sheet contains an overview of all open positions with basic information as average purchase price. The three fields "Stop Preis", "Limit Preis" and "Strategie" are for notes, which are kept at each execution of the script, if the corresponding position is still in the portfolio.

## Scope
*   optimized for Google Colab (https://colab.research.google.com/)
*   export optimized for "Portfolio Performance" (https://www.portfolio-performance.info/) and "Investing.com" (https://de.investing.com/)
*   for further information about the import at "Investing.com", see: https://www.investing-support.com/hc/en-us/articles/360000265217-Import-Portfolio-Holdings


# b) Configuration
## G-Drive Path
*   TradeRepublic statements will be imported via GoogleDrive. All PDF files shoule be in a single folder. You need to configure the path to your G-Drive before usage (see: "Configurations - to be defined by user")

# c) Options to customize
- for different data sources, see: https://colab.research.google.com/notebooks/io.ipynb
- in order to create different data structures, take a look at "Examples for extracted fields".

# d) Further Information
## Handling of costs statements
*   "Kosten des Wertpapierkaufs/verkaufs" are be considered.
*   "Kosten während der Haltedauer (pro Jahr)" are not extracted and therefore do not appear the sheets.

## Deposits & Withdrawls
* deposits and withdrawals to the depot are not recorded in PDF format. Therefore they are not taken into account and must be entered by hand if necessary.

## "Order" in Trade Republic documents
* each TradeRepublic document has got a "order" number. This is extracted and stored in the field "Notiz". It serves to prevent duplicate entries.  

# e) Backlog
## Open
* create sheets for portfolio performance in .csv format
* offer alternatives to G-Drive import/export

## To be fixed
- ...

# f) Changelog
## 0.1
* extract G-Drive folder of TradeRepublic PDFs
* create data structure (for Portfolio Performance or other purposes)
* generate master sheet of all transactions
* generate delta sheet for new transactions (base for TradeRepublic import)

## 0.2
- fixed | sort extracted transactions by date
- fixed | double entries of table labels in delta sheet

## 0.3
- add ticker symbol via finnhub api (requires your own api key)
- fixed | calculate purchase price
- add values to sheet ("Kaufkurs", "Gebühren")
- data structure for Investing.com import

## 0.4
- full refactoring (new class structure)
- add current portfolio as output option
- allow data enrichment of extracted data
- note already extracted PDFs

In [425]:
import re

#@markdown ### PDF Source
input_source = "G-Drive" #@param ["G-Drive"] {allow-input: true}

#@markdown ### GDrive Folder
# GDRIVE
# path to gdrive folder with your TradeRepublic pdfs

# e..g "/content/drive/My Drive/MY_TR_FOLDER/"
path_no_gdrive = "/content/drive/My Drive/MY_TR_FOLDER/"
path_gdrive = "/content/drive/My Drive/Documents/Investment/TradeRepublic/" #@param {type:"string"}

#@markdown ### Output Destination

output_destination = "Google-Sheets" #@param ["Google-Sheets"] {allow-input: true}

#@markdown ### Data Enrichment by OnvistaAPI (optional)
# formatted name of asset
# ticker symbol
# wkin
# asset type

onvista = False #@param {type:"boolean"}

#@markdown ### Optimized for Investing.com or Porfolio Performance

export_optimized_for = "Portfolio Performance" # @param ["Portfolio Performance","Investing.com","Finanzfluss"] {"allow-input":true}


#@markdown ### Create overview table of current portfolio (next to Delta & Master)

portfolio_sheet = True #@param {type:"boolean"}

#@markdown ### Create csv for Finanfluss

finanzfluss_csv = True #@param {type:"boolean"}

config = {
    "input_source" : input_source,
    "output_destination" : output_destination,
    "path_gdrive" : path_gdrive,
    "path_gdrive_id" : path_gdrive_id,
    "onvista" : onvista,
    "export_optimized_for" : export_optimized_for,
    "portfolio_sheet" : portfolio_sheet,
    "finanzfluss_csv" : finanzfluss_csv
}

#regex pattern
pdate = r"\d{2} \w{3}\. \d{4}"
pmoney = r"\s+\-?\d{1,3}(?:\.\d{3})*,\d{2}\s€"



In [426]:
# @title Basic Objects
## @title AccountStatement
class AccountStatement:
    def __init__(self, cashkonto, transactions):
        self.cashkonto = cashkonto
        self.transactions = transactions

    def __str__(self):
        # Converts the account statement to a readable string format
        cashkonto_info = f"Cashkonto: {self.cashkonto}"
        transactions_info = "\n".join([str(transaction) for transaction in self.transactions])
        return f"{cashkonto_info}\nTransactions:\n{transactions_info}"

class CashAccount:
    def __init__(self, data):
        cashkonto_data = data.replace("Cashkonto", "").strip()
        cashkonto_parts = cashkonto_data.split(" €")
        self.startBalance = helper.convert_german_euro_to_float(cashkonto_parts[0])
        self.incoming = helper.convert_german_euro_to_float(cashkonto_parts[1])
        self.outgoing = helper.convert_german_euro_to_float(cashkonto_parts[2])
        self.endBalance = helper.convert_german_euro_to_float(cashkonto_parts[3])

    def __str__(self):
        # Converts the cash account to a readable string format
        return (f"CashAccount - Startbalance: {self.startBalance}, Incoming: {self.incoming}, "
                f"Outgoing: {self.outgoing}, Endbalance: {self.endBalance}")

class Transaction:
    def __init__(self, data, cash):
      match = re.match(f"({pdate})(.*?)({pmoney}$)", data)
      if match:
        self.date = helper.convert_str_to_date(match.group(1).strip())
        rest = match.group(2).strip()
        endBalance = helper.convert_german_euro_to_float(match.group(3).strip())
        self.delta = endBalance - cash.startBalance
        deltastr = helper.convert_float_to_german_decimal(abs(self.delta))
        rest = rest.replace(f"{deltastr}", "")
        self.isin = helper.get_isin(rest)
        self.quantity = 0
        self.taType = "Transfer"
        if self.isin:
          rest = re.sub(f".*?{self.isin}", "", rest).strip()
          match = re.match(r"(.*?), quantity: (\d*.\d*)", rest)
          if "ETF" in rest:
            self.taType = "ETF"
          else:
            self.taType = "Stock"
          if match:
            rest = match.group(1)
            self.quantity = match.group(2)
            self.id = f"{self.isin}{helper.convert_date_to_sorting_string(self.date)}{self.quantity}"
        self.description = rest
        cash.startBalance = endBalance


    def __str__(self):
        # Converts the transaction to a readable string format
        return (f"Transaction - Date: {self.date}, Type: {self.taType} ISIN: {self.isin}, Description: {self.description}, "
                f"Quantity: {self.quantity}, Delta: {helper.convert_float_to_german_decimal(self.delta)}")

    def to_dict(self):
      isRich = config["onvista"]
      return {
            "ID": self.id,
            "Date": self.date,
            "Name": self.name if isRich else self.description,
            "Symbol": self.symbol if isRich else "",
            "Type": self.taType,
            "ISIN": self.isin,
            "Description": self.description,
            "Quantity": self.quantity,
            "Price": self.price,
            "Delta": self.delta,
        }

    def to_finanzfluss_csv(self):
      # return in this format Date;ISIN;Name;Type;Transaction;Delta;Quantity;Fee;Tax;Currency;ExchangeRate
      taType = "Aktie"
      if self.taType == "ETF":
        taType = "ETF"
      ta = "Kauf"
      if self.delta > 0:
        ta = "Verkauf"
      date = self.date.strftime("%d.%m.%Y")
      price = helper.convert_float_to_german_decimal(abs(self.price))
      quantity = self.quantity.replace(".", ",")
      return [date, self.isin, self.name if config["onvista"] else self.description, taType, ta, price, quantity, '0', '0', 'EUR', '1']

# Helper Methods


In [427]:
class HelperMethods:

  def __init__(self):
    # requirement for get_date_today(), convert_str_to_date()
    from datetime import date, datetime
    # pandas
    import pandas as pd
    #Install gsread
    # authenticate gsheet access
    # Load Google Drive helper
    from google.colab import drive
    from google.colab import auth

    try:
      import gspread
      from google.auth import default
      import PyPDF2
      from pyonvista import PyOnVista
    except:
      self.install_libraries()
      import gspread
      from google.auth import default
      import PyPDF2
      from pyonvista import PyOnVista

    self.date = date
    self.datetime = datetime
    self.pd = pd
    self.gspread = gspread
    self.auth = auth
    self.default = default
    self.drive = drive

  def install_libraries(self):
    !pip install --upgrade --quiet gspread
    !pip install --upgrade --quiet google-auth
    !pip install PyPDF2
    !pip install pyonvista

  def get_access_credentials_for_gspread(self):
    helper.auth.authenticate_user()
    creds, _ = helper.default() # this line was added
    # get access credentials for gsheet
    return helper.gspread.authorize(creds) # this line was changed

  def get_spreadsheet(self, spreadsheet_name):
    gc = self.get_access_credentials_for_gspread()
    config[spreadsheet_name] = True
    try:
      return gc.open(spreadsheet_name)
    except:
      config[spreadsheet_name] = False
      return gc.create(spreadsheet_name)

  def get_date_today(self):
    today = self.date.today()
    return self.convert_date_to_sorting_string(today)

  def convert_str_to_date(self, input_string):
    return self.datetime.strptime(input_string, "%d %b. %Y").date()

  def convert_date_to_str(self, input_date):
    output = input_date.strftime("%d %b. %Y")
    return output

  def convert_date_to_sorting_string(self, input_date):
    return input_date.strftime("%Y%m%d")

  def convert_german_decimal_to_float(self, german_decimal_str):
    # input: 20.000,00; output: 20000.00
    return float(german_decimal_str.replace(".", "").replace(",", "."))

  def convert_float_to_german_decimal(self, number):
    return '{:_.2f}'.format(number).replace(".", ",").replace("_", ".")

  def convert_german_euro_to_float(self, euro_str):
    return self.convert_german_decimal_to_float(euro_str.replace("€", "").strip())

  def convert_float_to_german_euro(self, number):
    return self.convert_float_to_german_decimal(number) + " €"

  def calculate_weighted_average(self, dataframe, column_for_average, column_for_weight):
    a = dataframe[column_for_average]
    w = dataframe[column_for_weight]
    # calulate average
    weighted_average = (a * w).sum() / w.sum()
    # convert float to german decimal
    return weighted_average

  def abs_sum_dataframe_with_german_decimals(self, dataframe_column):
    df = abs(dataframe_column.sum()) # absolute sum
    return df

  def get_diff_between_2_dataframes(self, df1, df2, which=None):
    """
      Find Rows Which Are Not common Between Two dataframes - by subset "ID"
        https://kanoki.org/2019/07/04/pandas-difference-between-two-dataframes/

      Remember:
        If "ID" was not used as a subset to filter,
        it would no longer be possible to edit and complete the sheet (notes, WKN or other missing information).
        All changes would be overwritten with the next execution of the script.
    """
    return self.pd.concat([df1,df2]).drop_duplicates(subset=["ID"], keep=False)

  def sort_dataframe_column_by_label(self, my_dataframe, label):
    return my_dataframe.reindex(columns=label)

  def sort_dataframe_rows_by_date(self, my_dataframe):
    # convert to str to datetime
    my_dataframe["Date"] = self.pd.to_datetime(my_dataframe["Date"])
    # sort by date
    my_dataframe = my_dataframe.sort_values(by = "Date")
    # change format
    my_dataframe["Date"] = my_dataframe["Date"].dt.strftime("%d.%m.%Y")
    # convert datetime to str
    my_dataframe["Date"] = my_dataframe["Date"].astype(str)
    # remove NaT artefacts after conversion
    my_dataframe = my_dataframe[my_dataframe.Date != "NaT"]
    return my_dataframe

  def get_isin(self, text_body):
    """TradeRepublic uses ISIN only - ticker symbol or WSIN have to be added in later processing"""
    isin_pattern = "((?:BE|BM|FR|BG|VE|DK|HR|DE|JP|HU|HK|JO|BR|XS|FI|GR|IS|RU|LB|PT|NO|TW|UA|TR|LK|LV|LU|TH|NL|PK|PH|RO|EG|PL|AA|CH|CN|CL|EE|CA|IR|IT|ZA|CZ|CY|AR|AU|AT|IN|CS|CR|IE|ID|ES|PE|TN|PA|SG|IL|US|MX|SK|KR|SI|KW|MY|MO|SE|GB|GG|KY|JE|VG|NG|SA|MU)(?:[0-9A-Z]{9})(?:[0-9]))"
    matches = re.findall(isin_pattern, text_body)
    if matches:
      return matches[0]
    return ""

# Input
- defines the source of the PDFs to be extracted

In [428]:
class Input:
  def __init__(self):
    if config["input_source"] == "G-Drive":
      self.connect_google_drive()

  def connect_google_drive(self):
    import os
    """option to use google drive as source"""
    # This will prompt for authorization
    helper.drive.mount('/content/drive/', force_remount=True)
    os.chdir(config["path_gdrive"])


# Sheet Source
- defines the source of the previously created sheets

In [429]:
class Sheet_Source:
  def __init__(self):
    self.gc = helper.get_access_credentials_for_gspread()
    # g sheet as output & source of existing sheets
    if config["output_destination"] == "Google-Sheets":
      self.get_extracted_pdf()

  def get_extracted_pdf(self):
    try: # yes => need to check which data is new and append
      spreadsheet = self.gc.open("TradeRepublic_Extracted_Files")
      worksheet_extracted = spreadsheet.sheet1 # get worksheet for document
      extracted_sheet = self.pd.DataFrame(worksheet_extracted.get_all_records())

      config["list_extracted_pdfs"] = extracted_sheet
    except: # no => you can just fill in the extracted data as master sheet
      config["list_extracted_pdfs"] = []

# Extract PDFs
- performs the extraction of the PDFs
- only previously unknown documents are extracted

In [430]:
class ExtractPdfs:
  def __init__(self):
    # PyPDF2 for PDF extraction
    # requirement for pdf-folder extraction
    import os
    import glob
    import PyPDF2

    self.PyPDF2 = PyPDF2
    self.glob = glob
    self.os = os

    # config G-Drive
    if config["input_source"] == "G-Drive" and (config["path_gdrive"] == path_no_gdrive or config["path_gdrive"] == ""):
        print("Please provide the Google Drive path for your TradeRepublic PDFs folder")
    else:
      self.path = config["path_gdrive"]

  def start(self):
    """
      Checks for relevant PDF in folder and extracts full text
        Parameters
        ----------
        path : str
            relative path for folder

        Output
        ------
        raw_text_list : list
          A list for all raw pdf extractions - one item is one pdf
          [
            "Wertpapierordenummer\n\...",
            ...
          ]
    """
    raw_text_list = []
    for filename in self.glob.glob(self.os.path.join(self.path, '*.pdf')):
      with open(filename, 'rb') as fin: # open in readonly mode

        # extract only pdfs which are not extracted already
        if filename in config["list_extracted_pdfs"]:
          print("already existed", filename)
          continue
        else:
          config["list_extracted_pdfs"].append(filename)

        # read and extract pdf infos
        pdf_reader = self.PyPDF2.PdfReader(fin)
        # extract first page
        isValid = False
        doc = ""
        for extr_page in pdf_reader.pages:
          text = extr_page.extract_text()
          if not isValid:
            # relevance criteria
            if ")." in filename: # try to find pdf duplicates - eg. "filename (1).pdf" instead of "filename.pdf"
                print("Processing PDF | DUPLICATE   | \"WERTPAPIERABRECHNUNGÜBERSICHT\" |  " + filename)
                break
            elif "UMSATZÜBERSICHT" in text:  ## all non duplicate,"Wertpapergeschäftsorder"
                print("Processing PDF | RELEVANT    | \"Account Statement\"             |  " + filename)
                isValid = True
                pass
            elif "DEPOTAUSZUG" in text: ## securities, not important yet
                print("Processing PDF | IRELEVANT   | \"Securities\"                    |  " + filename)
                break
            else: # non duplicate, irelevant files
                print("Processing PDF | IRELEVANT   |                                   |  " + filename)
                break
          #remove footer Seite   von 3 3
          doc = doc + re.sub(f"Trade Republic Bank GmbH.*Seite   von \d+ \d+|{pdate} - {pdate}|Zum {pdate}", " ", text, flags=re.DOTALL)
        # Regular expression to match dates "08 Apr. 2025"
        #entry_pattern = r"((?:Cashkonto|Deutsche Bank|Deutsche Managed Euro Fund|\d{2} \w{3}\. \d{4}).*?(?:\s+\-?\d{1,3}(?:\.\d{3})*,\d{2}\s€)+)"
        entry_pattern = f"((?:Cashkonto|{pdate}.*?(?:berweisung|Handel)).*?(?:{pmoney})+)"
        # Replace new lines and match €
        cleaned_text = re.sub(r'(\s*\n\s*|\xa0)', ' ', doc).replace("\u20AC", "€")
        entries = re.findall(entry_pattern, cleaned_text)

        for entry in entries:
          raw_text_list.append(entry)
    return raw_text_list

# Process Text
- processes the extracted PDFs into a temporary data structure

In [431]:
class ProcessText:
  def __init__(self, raw_text_list):
    cashAccount = {}
    self.transactions = []
    for item in raw_text_list:
      if "Cashkonto" in item:
        cashAccount = CashAccount(item)
      else:
        transaction = Transaction(item, cashAccount)
        if transaction.taType != "Transfer":
          self.transactions.append(transaction)

  def isin(self, text_body):
    """TradeRepublic uses ISIN only - ticker symbol or WSIN have to be added in later processing"""
    isin_pattern = "(BE|BM|FR|BG|VE|DK|HR|DE|JP|HU|HK|JO|BR|XS|FI|GR|IS|RU|LB|PT|NO|TW|UA|TR|LK|LV|LU|TH|NL|PK|PH|RO|EG|PL|AA|CH|CN|CL|EE|CA|IR|IT|ZA|CZ|CY|AR|AU|AT|IN|CS|CR|IE|ID|ES|PE|TN|PA|SG|IL|US|MX|SK|KR|SI|KW|MY|MO|SE|GB|GG|KY|JE|VG|NG|SA|MU)([0-9A-Z]{9})([0-9])"
    matches = re.findall(isin_pattern, text_body )
    item = "".join(matches[0])
    return item

  def name(self, text_body):
    pattern = "(?<=KURSBETRAG)(.*?)(?=ISIN)"
    name = re.findall(pattern, text_body)
    return name[0]

  def amount(self, text_body):
    ISIN = self.isin(text_body)
    pattern = "(?<=" + ISIN + ")(.*?)(?= Stk.)"
    amount = re.findall(pattern, text_body)
    return amount[0]

  def type(self, text_body):
    pattern = "(Kauf|Verkauf)"
    order_type = re.findall(pattern, text_body)
    return order_type[0]

  def value(self, text_body):
    currency = self.currency(text_body)
    pattern = "(?<=GESAMT)(.*)(?= "+ currency +"ABRECHNUNGPOSITIONBETRAG)"
    value = re.findall(pattern, text_body)
    return value[0]

# Enrich Data
- - enriches the data structure with additional information from third-party sources or calculations.

In [432]:
class EnrichData:
  def __init__(self, transactions):
    import requests
    import nest_asyncio
    self.requests = requests
    self.transactions = transactions
    nest_asyncio.apply()

  def start_enrichment_by_config(self):
    if config['onvista']:
        import asyncio
    for transaction in self.transactions:
      if transaction.taType == "Transfer":
        continue
      # add order price for investing.com import
      transaction.price = abs(transaction.delta)/float(transaction.quantity)

      # overwrite name, add wkin, ticker-symbol and asset type by onvista api
      if config["onvista"]:
        asyncio.run(self.information_by_onvista_api(transaction))

  async def information_by_onvista_api(self, transaction):
    """
      Enriches stock information by onvista
        Parameters
        ----------
        order_dict : dict
          {
              'Datum': '16.06.2020',
              'Wertpapiername': 'Alibaba',
              ...
          }

        Change in order_dict
        ------
        Wertpapiername : str
          Name of Stock
        Ticker-Symbol : str
          International Symbol (crucial for stock data)
        WKN : str
          Wertpapierkennnummer
        Vermögenswert : str
          Type of Asset

    """
    import aiohttp
    from pyonvista import PyOnVista
    async with aiohttp.ClientSession() as client:
      api = PyOnVista()
      await api.install_client(client)

      # Search for an instrument by name or ISIN
      instruments = await api.search_instrument(transaction.isin)  # Example: Volkswagen
      instrument = await api.request_instrument(instruments[0])

      # Print instrument details
      transaction.name = instrument.name
      transaction.symbol = instrument.symbol
      transaction.assettype = instrument.type


# Data Integration
- prepares the export to the target source
- extends the Mastersheet
- determines the delta sheet (new transactions)
- determines the current portfolio and the shares of the positions held in it

In [433]:
class DataIntegration:
  def __init__(self, transactions):
    # provides extractehelper- list of dictionaries
    self.transactions = transactions
    # pandas
    import pandas as pd
    self.pd = pd

    # CONFIGURATIONS
    self.master_and_delta_labels = ["ID", "Date", "Type", "Name", "Quantity",	"Price",	"Delta",	"ISIN", "Symbol", "Description"]

    # do you need a portfolio sheet next to master & delta sheet
    if config["portfolio_sheet"]:
      self.portfolio_labels = ["Date", "ISIN","Type", "Symbol", "Name", "TotalCost", "Quantity", "Ø Price"]

    # g sheet as output
    if config["output_destination"] == "Google-Sheets":
      self.gc = helper.get_access_credentials_for_gspread()

  def get_dataframes_for_gsheet_export(self):
    """
    Returns rows which need to be added to delta & master sheet
      checks which entries are new & puts them into dataframe for export

      Output
        ------
        data_update_sheets : dict
          Dictionary of dataframes which can be exported to master and delta sheet, eg.
          {
            "master" : DataframeForMasterUpdate,
            "delta" : DataframeForDeltaUpdate,
            "portfolio" : ""
          }
    """
    # check if sheets are exisiting (write result in config) & get existing data from sheets
    dataframe_master = self.get_gsheet_master()
    dataframe_delta = self.get_gsheet_delta()
    data_update_sheets = {
          "master" : dataframe_master,
          "delta" : dataframe_delta
      }

    # append portfolio if needed
    if config["portfolio_sheet"]:
      dataframe_portfolio = self.get_gsheet_portfolio(dataframe_master)
      # config for sheet update
      data_update_sheets["portfolio"] = dataframe_portfolio

    return data_update_sheets

  def get_extracted_data(self):
    """Converts extracted & enriched data (list of dictionaries) into a pandas dataframe"""
    extracted_data_dataframe = self.pd.DataFrame([transaction.to_dict() for transaction in self.transactions])
    return extracted_data_dataframe

  def get_gsheet_master(self):
    # get new data from pdfs
    extracted_data = self.get_extracted_data()

    filename = "TradeRepublic_Master"
    spreadsheet = helper.get_spreadsheet(filename)
    # check if master sheet exists
    if config[filename]:
      worksheet_master = spreadsheet.sheet1 # get worksheet for document
      master_sheet = self.pd.DataFrame(worksheet_master.get_all_records())

      # get what new data do you have
      new_dataframe = helper.get_diff_between_2_dataframes(master_sheet, extracted_data)
      # delta => save difference for delta sheet
      self.new_delta = new_dataframe

      # Filter out empty or all-NA columns before concatenation
      master_sheet = master_sheet.dropna(axis=1, how='all')
      new_dataframe = new_dataframe.dropna(axis=1, how='all')

      # append the new data to master sheet
      new_dataframe = self.pd.concat([master_sheet, new_dataframe], ignore_index=True)
      # update config file
    else: # no, you can just fill in the extracted data as master sheet
      new_dataframe = extracted_data
      # delta => save difference for delta sheet
      self.new_delta = extracted_data

    # sort columns by label
    dataframe_master = helper.sort_dataframe_column_by_label(new_dataframe, self.master_and_delta_labels)
    #remove "NAN"
    dataframe_master.fillna('', inplace=True)
    # sort rows (by date)
    dataframe_master  = helper.sort_dataframe_rows_by_date(dataframe_master)

    return dataframe_master

  def get_gsheet_delta(self):
    # get new data from pdfs
    extracted_data = self.get_extracted_data()
    # check if delta sheet exists
    today = helper.get_date_today()
    filename = "TradeRepublic_Delta_" + today
    spreadsheet = helper.get_spreadsheet(filename)
    if config[filename]: # yes => need to check which data is new does not exist in current sheet already
      worksheet_delta = spreadsheet.sheet1
      delta_sheet = self.pd.DataFrame(worksheet_delta.get_all_records())
      # was created in get_gsheet_master and represents the diference of the extracted data and the master sheet => the delta!
      new_delta = self.new_delta
      delta_sheet = delta_sheet.dropna(axis=1, how='all')
      new_delta = new_delta.dropna(axis=1, how='all')
      # append the new data to delta sheet
      dataframe_delta = self.pd.concat([delta_sheet, new_delta], ignore_index=True)
    else:
      # was created in get_gsheet_master and represents the diference of the extracted data and the master sheet => the delta!
      dataframe_delta = self.new_delta
    # sort column by label
    dataframe_delta = helper.sort_dataframe_column_by_label(dataframe_delta, self.master_and_delta_labels)
    #remove "NAN"
    dataframe_delta.fillna('', inplace=True)
    # sort rows (by date)
    dataframe_delta  = helper.sort_dataframe_rows_by_date(dataframe_delta)
    return dataframe_delta

  def get_gsheet_portfolio(self, dataframe_master):
    filename = "TradeRepublic_Portfolio"
    spreadsheet = helper.get_spreadsheet(filename)

    if config[filename]:
      # does the sheet already exists?
      worksheet_portfolio = spreadsheet.sheet1 # get worksheet for document
      exisiting_portfolio = self.pd.DataFrame(worksheet_portfolio.get_all_records())
      # prepare current portfolio
      new_portfolio = self.current_portfolio_by_master_sheet(dataframe_master)
      # merge extracted & new portfolio
      dataframe_portfolio = new_portfolio.merge(exisiting_portfolio["ISIN"], on='ISIN', how="left")
      dataframe_portfolio.fillna('', inplace=True)

    else:
      # portfolio does not exists so it can be created from scratch
      dataframe_portfolio = self.current_portfolio_by_master_sheet(dataframe_master)
      dataframe_portfolio.fillna('', inplace=True)

    return dataframe_portfolio

  def current_portfolio_by_master_sheet(self, dataframe_all_orders):
    """
      Calcultes the amount of stock in current porfolio and it's average purchase price
          Parameters
          ----------
          dataframe_all_orders : pd.dataframe
            all orders

          Output
          ------
          ----------
          dataframe_portfolio : pd.dataframe
            current holdings with key infrmation as average purchase price etc.
    """
    current_portfolio = self.pd.DataFrame( columns= self.portfolio_labels) # emtpy dataframe with correct labeling

    # filter down unique stocks by isin
    unique_stocks = dataframe_all_orders.drop_duplicates(subset=['ISIN'])
    # loop through unique stocks and handover to calculate the current holding in the portfolio
    for index, row in unique_stocks.iterrows():
      current_isin = row["ISIN"]
      orders_for_current_isin = dataframe_all_orders [ dataframe_all_orders['ISIN'] == current_isin ].copy() # filter down orders of current isin
      # calculate the current number of shares in the portfolio
      portfolio_current_stock = self.calculate_stock_in_portfolio( orders_for_current_isin )
      current_portfolio = current_portfolio.dropna(axis=1, how='all')
      portfolio_current_stock = portfolio_current_stock.dropna(axis=1, how='all')
      current_portfolio = self.pd.concat( [current_portfolio, portfolio_current_stock] ) # append to dataframe for to portfolio
    return current_portfolio


  def summarize_portfolio_orders(self, dataframe_orders):
    """
     Calculates on the basis of transactions how many shares are currently in the portfolio, their total value and average purchase price
      Parameters
      ----------
      dataframe_orders : pd.dataframe
        transaction for a single share value

      Output
      ------
      dataframe_stock_in_portfolio : pd.dataframe
        returns a single row in the "self.portfolio_labels" indexed dataframe
    """

    # calcualte weighted average for purchase price
    weighted_average = helper.calculate_weighted_average(dataframe_orders, "Price", "Quantity")
    # get the total amount of stocks
    order_amount = dataframe_orders["total_holdings"].iloc[-1] # cell in last row
    # sum value and turn into positive number "Gezahlter Wert" ~ or something similar
    order_value = helper.abs_sum_dataframe_with_german_decimals(dataframe_orders["Delta"])

    dataframe_stock_in_portfolio = dataframe_orders.tail(1).copy()
    dataframe_stock_in_portfolio["Quantity"] = order_amount
    dataframe_stock_in_portfolio["Ø Price"] = weighted_average
    dataframe_stock_in_portfolio["TotalCost"] = order_value
    # reindex with portfolio labels
    dataframe_stock_in_portfolio = dataframe_stock_in_portfolio.reindex(self.portfolio_labels, axis="columns")
    return dataframe_stock_in_portfolio

  def calculate_stock_in_portfolio(self, dataframe_orders_single_stock):
    """
      Checks the amount of stocks left for each share value
        if non => return
        if still stocks left & never completley sold => return calculated remaining shares
        if still stocks left & in between sold all => filter down relevant once, return calculated remaining shares


        Parameters
        ----------
        dataframe_orders_single_stock : pd.dataframe
          contains all orders for a specific stock

        Output
        ------
        "" : pd.dataframe
          returns dataframe for total holdings of a single share value
            if non left => return
            if still stocks left & never completley sold => return calculated remaining shares
            if still stocks left & in between sold all => filter down relevant once, return calculated remaining shares

    """
    # "change of stock amount" add the sign to the amount of stocks for each order; sell 5 => -5; buy 5 => 5;
    dataframe_orders_single_stock["Quantity"] = dataframe_orders_single_stock["Quantity"].astype(float)
    dataframe_orders_single_stock["change_of_stock_amount"] = dataframe_orders_single_stock.apply( lambda row : row["Quantity"] if row["Delta"] < 0 else -1 * row["Quantity"] , axis=1)

    # sum up current stocks in portfolio
    dataframe_orders_single_stock["change_of_stock_amount"] = dataframe_orders_single_stock["change_of_stock_amount"].astype(float)
    dataframe_orders_single_stock["total_holdings"] = dataframe_orders_single_stock["change_of_stock_amount"].cumsum()


    # check last row for current amount of stock in portfolio which contains the current holding of each stock
    last_row = dataframe_orders_single_stock.tail(1)
    if 0 in last_row["total_holdings"].values:
      # there are not stocks left anymore => kick out the stock
      return

    else:
      # Check if stock was sold out in between, but bought again into current portfolio
        # a) stock was never sold completley => all orders are relevant for calculation
      if 0 not in dataframe_orders_single_stock["total_holdings"].values:
        orders_for_current_portfolio =  self.summarize_portfolio_orders(dataframe_orders_single_stock)
        return orders_for_current_portfolio

        # b) stock was sold compleltly in between => orders should be filtered since last complete sell
      else:
        # filter to the last time the the stock was sold completely
          # Reverse the column C then use Series.ne + Series.cummin to create a boolean mask
        boolean_mask = dataframe_orders_single_stock.loc[::-1, "total_holdings"].ne(0).cummin()[::-1]
          # then use this mask to filter the rows in datframe:
        last_remaining = dataframe_orders_single_stock[boolean_mask]

        # calulate the current holdings based - only considering relevant oders in the books
        relevant_orders_for_current_portfolio = self.summarize_portfolio_orders(last_remaining)
        return relevant_orders_for_current_portfolio

# Output
- performs the export of the specified sheets

In [434]:
class Output:
  def __init__(self, dataframe, transactions):
    # get access credentials for gsheet
    self.gc = helper.get_access_credentials_for_gspread()
    import pandas as pd
    self.pd = pd
    import os
    self.os = os

    # update list of already extracted pdfs
    self.update_extracted_pdf()

    self.update_master(dataframe["master"])
    self.update_delta(dataframe["delta"])

    if config["portfolio_sheet"]:
      self.update_portfolio(dataframe["portfolio"])

    if config['finanzfluss_csv']:
      import csv, glob
      strDate = helper.convert_date_to_sorting_string(pd.to_datetime(dataframe["master"]["Date"], format='%d.%m.%Y').max())
      filename = os.path.join(config["path_gdrive"], f'finanzfluss_{strDate}.csv')

      with open(filename, mode="w", newline="", encoding="utf-8") as file:
          writer = csv.writer(file, delimiter=';')
          writer.writerow(["Datum","ISIN","Name","Typ","Transaktion","Preis","Anzahl","Gebühren","Steuern","Währung","Wechselkurs"])
          writer.writerows([transaction.to_finanzfluss_csv() for transaction in transactions])

  def update_extracted_pdf(self):
    filename = "TradeRepublic_Extracted_Files"
    extracted_pdfs_dataframe = self.pd.DataFrame(config["list_extracted_pdfs"])
    spreadsheet = helper.get_spreadsheet(filename)
    worksheet_extracted = spreadsheet.sheet1 # get worksheet for document
    if config[filename]:
      worksheet_extracted.clear() # alten Worksheet löschen

    worksheet_extracted.update(
        extracted_pdfs_dataframe.values.tolist() # value
    )

  def update_master(self, master_data):
    master_update_dataframe = self.pd.DataFrame(master_data)

    filename = "TradeRepublic_Master"
    spreadsheet = helper.get_spreadsheet(filename)
    if config[filename]:
      worksheet_master = spreadsheet.sheet1 # get worksheet for document
      worksheet_master.clear() # alten Worksheet löschen
    else:
      worksheet_master = spreadsheet.sheet1 # get worksheet for document

    worksheet_master.update(
        [master_update_dataframe.columns.values.tolist()] # label
        +
        master_update_dataframe.values.tolist() # value
    )

  def update_delta(self, delta_data):
    delta_update_dataframe = self.pd.DataFrame(delta_data)
    today = helper.get_date_today()
    filename = "TradeRepublic_Delta_" + today
    spreadsheet = helper.get_spreadsheet(filename)
    worksheet_delta = spreadsheet.sheet1 # get worksheet for document

    if config[filename]:
      worksheet_delta.clear() # alten Worksheet löschen

    worksheet_delta.update(
        [delta_update_dataframe.columns.values.tolist()] # label
        +
        delta_update_dataframe.values.tolist() # value
    )

  def update_portfolio(self, portfolio_data):
    portfolio_update_dataframe = self.pd.DataFrame(portfolio_data)
    filename = "TradeRepublic_Portfolio"
    spreadsheet = helper.get_spreadsheet(filename)
    worksheet_portfolio = spreadsheet.sheet1 # get worksheet for document

    if config[filename]:
      worksheet_portfolio.clear() # alten Worksheet löschen

    worksheet_portfolio.update(
        [portfolio_update_dataframe.columns.values.tolist()] # label
        +
        portfolio_update_dataframe.values.tolist() # value
    )

    def __str__(self):
      return "Output processing completed."

# Script Execution

In [435]:
helper = HelperMethods()

# a) define input source
Input()

# b) define sheet source
Sheet_Source()

# c) extract pdfs
raw_text_list = ExtractPdfs().start()

# d) process text
processed_info = ProcessText(raw_text_list)
transactions = processed_info.transactions

# e) enrich data
EnrichData(transactions).start_enrichment_by_config()

# f) integrate data
dict_data_for_export = DataIntegration(transactions).get_dataframes_for_gsheet_export()

# g) export sheets
output = Output(dict_data_for_export, transactions)


Mounted at /content/drive/
Processing PDF | RELEVANT    | "Account Statement"             |  /content/drive/My Drive/Documents/Investment/TradeRepublic/Account statement 250411.pdf
Processing PDF | IRELEVANT   | "Securities"                    |  /content/drive/My Drive/Documents/Investment/TradeRepublic/Securities 250411.pdf


In [436]:
dict_data_for_export["master"]

,ID,Date,Type,Name,Quantity,Price,Delta,ISIN,Symbol,Description
0,IE00B3YLTY66202504100.404669,10.04.2025,ETF,SPDR MSCI All Country World Investable Market ...,0.404669,205.649556,-83.22,IE00B3YLTY66,SPYI,SSGA SPDR ETFs Europe I plc - SPDR MSCI All Co...
1,IE00B3YLTY66202504103.0,10.04.2025,ETF,SPDR MSCI All Country World Investable Market ...,3.000000,205.983333,-617.95,IE00B3YLTY66,SPYI,SSGA SPDR ETFs Europe I plc - SPDR MSCI All Co...
2,US67066G1040202504100.8726,10.04.2025,Stock,Nvidia,0.872600,104.286042,-91.00,US67066G1040,NVD,"NVIDIA CORP. DL-,001"
3,US0231351067202504100.461414,10.04.2025,Stock,Amazon,0.461414,175.525667,-80.99,US0231351067,AMZ,"AMAZON.COM INC. DL-,01"
4,US0846707026202504100.084521,10.04.2025,Stock,Berkshire Hathaway B,0.084521,485.086546,-41.00,US0846707026,BRYN,BERKSHIRE HATHAWAY
5,US5949181045202504100.056609,10.04.2025,Stock,Microsoft,0.056609,370.789097,-20.99,US5949181045,MSF,"MICROSOFT DL-,00000625"
6,US0378331005202504100.391981,10.04.2025,Stock,Apple,0.391981,181.182251,-71.02,US0378331005,APC,APPLE INC.


In [437]:
dict_data_for_export["delta"]

,ID,Date,Type,Name,Quantity,Price,Delta,ISIN,Symbol,Description
0,IE00B3YLTY66202504100.404669,10.04.2025,ETF,SPDR MSCI All Country World Investable Market ...,0.404669,205.649556,-83.22,IE00B3YLTY66,SPYI,SSGA SPDR ETFs Europe I plc - SPDR MSCI All Co...
1,IE00B3YLTY66202504103.0,10.04.2025,ETF,SPDR MSCI All Country World Investable Market ...,3.000000,205.983333,-617.95,IE00B3YLTY66,SPYI,SSGA SPDR ETFs Europe I plc - SPDR MSCI All Co...
2,US67066G1040202504100.8726,10.04.2025,Stock,Nvidia,0.872600,104.286042,-91.00,US67066G1040,NVD,"NVIDIA CORP. DL-,001"
3,US0231351067202504100.461414,10.04.2025,Stock,Amazon,0.461414,175.525667,-80.99,US0231351067,AMZ,"AMAZON.COM INC. DL-,01"
4,US0846707026202504100.084521,10.04.2025,Stock,Berkshire Hathaway B,0.084521,485.086546,-41.00,US0846707026,BRYN,BERKSHIRE HATHAWAY
5,US5949181045202504100.056609,10.04.2025,Stock,Microsoft,0.056609,370.789097,-20.99,US5949181045,MSF,"MICROSOFT DL-,00000625"
6,US0378331005202504100.391981,10.04.2025,Stock,Apple,0.391981,181.182251,-71.02,US0378331005,APC,APPLE INC.


In [438]:
dict_data_for_export["portfolio"]

,Date,ISIN,Type,Symbol,Name,TotalCost,Quantity,Ø Price
0,10.04.2025,IE00B3YLTY66,ETF,SPYI,SPDR MSCI All Country World Investable Market ...,701.17,3.404669,205.943661
1,10.04.2025,US67066G1040,Stock,NVD,Nvidia,91.00,0.872600,104.286042
2,10.04.2025,US0231351067,Stock,AMZ,Amazon,80.99,0.461414,175.525667
3,10.04.2025,US0846707026,Stock,BRYN,Berkshire Hathaway B,41.00,0.084521,485.086546
4,10.04.2025,US5949181045,Stock,MSF,Microsoft,20.99,0.056609,370.789097
5,10.04.2025,US0378331005,Stock,APC,Apple,71.02,0.391981,181.182251
